In [9]:
import sys, os
if os.path.join(os.path.abspath(os.getcwd()), "utils") not in sys.path :
    sys.path.append(os.path.join(os.path.abspath(os.getcwd()), "utils"))

import pandas as pd
import numpy as np
import networkx as nx

import freud, random

from matplotlib import pyplot as plt

import crystal_utils as cu

In [10]:
## starting input .CONFIG file with atoms
prod_file = './templates/bdp_template_12101.CONFIG'  ## starting input file

## parameters: metal_input is in the starting file; metal_output gets written.
metal_input  = 'Ag'
metal_output = 'Na'

## defect atom, typically 'Mg'. 'Li' works too.
defect = 'Mg'

## rule for placing defects: 'unsym', 'symm', 'all', 'Al4'
rule = 'unsym'
enforce_d = True if rule == 'symm' else False

## exclude: minimum enforced distance between defects
# excl = 1 if defect == 'Li' else 0
excl = 0

## reset seed once for the whole run
random.seed(23)

In [11]:
## create a dummy output directory
to_dir = f'./{metal_output} {rule}_{excl} {defect}-doped/'
if not os.path.exists(to_dir): os.makedirs(to_dir)

## read the input .CONFIG file
phase, intro_line, cell, atoms = cu.read_poly(prod_file, fractional=False)

## set a shorthand, this is only for fractional=False just above.
Lx = cell[0, 0]
Ly = cell[1, 1]
Lz = cell[2, 2]

## find the dimensions of the file. 
dimension = [5.630, 9.7514/2, 33.45]
x_dimension = cell[0, 0] / dimension[0]
y_dimension = cell[1, 1] / dimension[1]
z_dimension = cell[2, 2] / dimension[2]

# print(f"Input file has dimensions (UC): {x_dimension:.1f} x {y_dimension:.1f} x {z_dimension:.1f}")
# print(f"Input file has dimensions (AA): {cell[0, 0]} x {cell[1, 1]} x {cell[2, 2]}")

## Find all z-coordinates for conduction planes in the input file: this outputs a list of z-coordinates.
## The z coordinate is now either fractional (from -0.5 to +0.5) or real (angstroms)
planes = cu.get_conduction_planes(atoms, metal_input, inexact=True)

## Find the number of Al atoms that will be randomly substituted by Mg.
## num_sub_al = "Number of (will be) substituted aluminium".
num_defects_to_be = int( cu.find_no_sub_al(atoms, x= 1/3 if defect == 'Li' else 2/3) / len(planes) )

## add symmetry to the atoms, e.g. Al1, Al2, etc.
new_atoms = cu.add_symmetry_bdp(atoms,cell, metal_input, frac=False)
al4_atoms = new_atoms.query('symm == "Al4"')
al2_atoms = new_atoms.query('symm == "Al2"')

Input file has dimensions (UC): 10.0 x 12.0 x 1.0
Input file has dimensions (AA): 56.3 x 58.5084 x 33.45
